# 从零开始构建你的第一个神经网络

在这个笔记本中，您将实现构建深度神经网络所需的所有函数。

**完成此lab后，你可以实现如下功能:**
- 构建一个更深的神经网络（具有超过1个隐藏层）
- 实现一个易于使用的神经网络类

**符号说明**：
- 上标 $[l]$ 表示与第 $l$ 层相关的量。
    - 例如：$a^{[L]}$ 是第 $L$ 层的激活值。$W^{[L]}$ 和 $b^{[L]}$ 是第 $L$ 层的参数。
- 上标 $(i)$ 表示与第 $i$ 个示例相关的量。
    - 例如：$x^{(i)}$ 是第 $i$ 个训练样本。
- 下标 $i$ 表示向量的第 $i$ 个条目。
    - 例如：$a^{[l]}_i$ 表示第 $l$ 层激活值的第 $i$ 个条目。
    
让我们开始吧！！！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

# 1 - 全连接（仿射）层

## 1.1 - 线性前向传播

您将从实现一些基础函数开始，稍后在实现模型时将会用到这些函数。您需要按照以下顺序完成三个函数：

- LINEAR
- LINEAR -> ACTIVATION，其中 ACTIVATION 将是 ReLU 或 Sigmoid 中的一个。
- [LINEAR -> RELU] × (L-1) -> LINEAR -> SIGMOID（完整模型）

线性前向模块（针对所有示例的向量化）计算以下方程式：

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

其中 $A^{[0]} = X$。

**练习**：构建前向传播的线性部分。请实现 `layers.py`中的 `linear_forward` 函数。

**提示**：
这个单元的数学表示是 $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$。您可能会发现 `np.dot()` 函数很有用。如果您的维度不匹配，打印 `W.shape` 可能会有帮助。


In [ ]:
from layers import linear_forward
from test_cases import linear_forward_test_case

A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

**预期得到的输出**:

```
Z = [[ 3.26295337 -1.23429987]]
```

## 1.2 - 线性反向传播

对于第 $l$ 层，线性部分是：$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$（其后是激活函数）。

假设您已经计算出了导数 $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$。您想要得到 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$。

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **图 4** </center></caption>

三个输出 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ 使用输入 $dZ^{[l]}$ 来计算。以下是您需要的公式：
$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$

在执行线性反向传播时，您将使用这些公式来计算梯度。在实际的代码实现中，您将使用这些导数来更新参数（权重 $W^{[l]}$ 和偏置 $b^{[l]}$），以及计算前一层的梯度 $dA^{[l-1]}$，这对于实现多层网络中的反向传播至关重要。

**练习**：请实现 `layers.py`中的 `linear_backward` 函数。

**提示**：
您可能会发现 `np.dot()` 和 `np.sum()` 函数很有用。如果您的维度不匹配，打印 `W.shape` 可能会有帮助。


In [ ]:
from layers import linear_backward
from test_cases import linear_backward_test_case

dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print("dA_prev = "+ str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db))

**预期得到的输出**:
    
```
dA_prev = 
 [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = 
 [[-0.10076895  1.40685096  1.64992505]]
db = 
 [[0.50629448]]
```

# 2 - 激活函数

## 2.1 - Sigmoid 函数

### 2.1.1 - Sigmoid 前向传播

**Sigmoid 函数**：$\sigma(Z) = \sigma(W A + b) = \frac{1}{1 + e^{-(W A + b)}}$。

**练习**：请实现 `layers.py`中的 `sigmoid_forward` 函数。

**提示**：我们为您提供了 `sigmoid_forward` 函数。这个函数返回**两个**项：激活值 "`a`" 和一个包含 "`Z`" 的 "`cache`"（这是我们将输入到相应的反向函数中的内容）。您可能会发现 `np.exp()` 函数很有用。


In [ ]:
from layers import sigmoid_forward
from test_cases import sigmoid_forward_test_case

Z = sigmoid_forward_test_case()

A, cache = sigmoid_forward(Z)

print("A = " + str(A))
print("cache = " + str(cache))

**预期得到的输出**:
    
```
A = [[0.39729283 0.485937   0.10562821]
 [0.83757178 0.14265203 0.3011669 ]]
cache = [[-0.41675785 -0.05626683 -2.1361961 ]
 [ 1.64027081 -1.79343559 -0.84174737]]
```

### 2.1.2 - Sigmoid backward

Sigmoid 函数：$\sigma(Z) = \frac{1}{1 + e^{-Z}}$。

其导数（对z的梯度）可以表示为：$\frac{d\sigma(z)}{dz} = \sigma(z) (1 - \sigma(z))$

**练习**：请实现 `layers.py`中的 `sigmoid_backward` 函数。

**提示**：您可能会发现 `np.exp()` 函数很有用。

In [ ]:
from layers import sigmoid_backward
from test_cases import sigmoid_backward_test_case

dA, cache = sigmoid_backward_test_case()

dZ = sigmoid_backward(dA, cache)

print("dZ = " + str(dZ))

**预期得到的输出**:
    
```
dZ = dZ = [[-0.09787036 -0.00976551 -0.40863549]
 [ 0.335792   -0.41592828 -0.07015842]]
```

## 2.2 Relu 函数

### 2.2.1 Relu 函数前向传

**ReLU**：ReLU的数学公式是 $A = RELU(Z) = \max(0, Z)$。我们为您提供了 `relu` 函数。这个函数返回**两个**项：激活值 "`A`" 和一个包含 "`Z`" 的 "`cache`"（这是我们将输入到相应的反向函数中的内容）。

**练习**：请实现 `layers.py`中的 `relu_forward` 函数。

**提示**：您可能会发现 `np.maximum()` 函数很有用。

In [ ]:
from layers import relu_forward
from test_cases import relu_forward_test_case

Z = relu_forward_test_case()

A, cache = relu_forward(Z)

print("A = " + str(A))
print("cache = " + str(cache))

**预期得到的输出**:
    
```
A = [[1.62434536 0.         0.        ]
 [0.         0.86540763 0.        ]]
cache = [[ 1.62434536 -0.61175641 -0.52817175]
 [-1.07296862  0.86540763 -2.3015387 ]]
```

### 2.2.2 Relu 函数后向传播


$$
\frac{d}{dz}ReLU(z)  = 
  \begin{cases} 
   0 & \text{if } z \le 0 \\
   1 & \text{if } z > 0 
  \end{cases}
$$

**练习**：请实现 `layers.py`中的 `relu_backward` 函数。

**提示**：您可能会发现 `np.array(_, copy=True) ` 函数很有用。

In [ ]:
from layers import relu_backward
from test_cases import relu_backward_test_case

dA, cache = relu_backward_test_case()

dZ = relu_backward(dA, cache)

print("dZ = " + str(dZ))

**预期得到的输出**:
    
```
dZ = [[ 1.62434536  0.         -0.52817175]
 [ 0.          0.86540763  0.        ]]
```

# 3 - 线性-激活层

## 3.1 - 前向传播

为了更加方便，您将会把两个函数（线性和激活）合并为一个函数（LINEAR->ACTIVATION）。因此，您将实现一个函数，它先进行线性前向步骤，然后是激活前向步骤。

**练习**：实现 *线性->激活* 层的前向传播。数学关系式为：$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$，其中激活函数 "g" 可以是 sigmoid 或 relu。请使用您之前实现的函数来实现`layers.py`中的 `linear_activation_forward` 函数。

In [ ]:
from layers import linear_activation_forward
from test_cases import linear_activation_forward_test_case

A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

**预期得到的输出**:
    
```
With sigmoid: A = [[ 0.96890023  0.11013289]]
With ReLU: A = [[ 3.43896131  0.        ]]
```

## 3.2 - 反向传播

如果 $g(.)$ 是激活函数，
`sigmoid_backward` 和 `relu_backward` 计算 $dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$。

**练习**：实现 *线性->激活* 层的反向传播。请实现`layers.py`中的 `linear_activation_backward` 函数

In [ ]:
from layers import linear_activation_backward
from test_cases import linear_activation_backward_test_case


dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

**预期得到的输出**:
    
```
sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]
```

# 4 - L-Layer Model 

## 4.1 - 前向传播


为了在实现 \(L\) 层神经网络时更加方便，你需要一个函数，这个函数复制前面的函数（`linear_activation_forward` 与 Relu）\(L-1\) 次，然后跟随一个 `linear_activation_forward` 与 Sigmoid。

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **图 2** : *[线性 -> Relu] \( \times (L-1) \) -> 线性 -> Sigmoid* 模型</center></caption><br>

**练习**：请您实现`layers.py`中的 ` L_model_forward` 函数。

**说明**：在下面的代码中，变量 `AL` 将表示 \(A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})\)。（这有时也被称为 `Yhat`，即这是 \(\hat{Y}\)。）

**提示**：
- 使用你之前写过的函数
- 使用 for 循环复制 [线性->RELU] \(L-1\) 次
- 不要忘记在 "caches" 列表中跟踪缓存。要在 `list` 中添加一个新值 `c`，可以使用 `list.append(c)`。

In [ ]:
from layers import L_model_forward
from test_cases import L_model_forward_test_case_2hidden

X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

**预期得到的输出**:
    
```
AL = [[ 0.03921668  0.70498921  0.19734387  0.04728177]]
Length of caches list = 3
```

## 4.2 - 后向传播

与前向传播一样，您将实现反向传播的辅助函数。请记住，反向传播用于计算损失函数相对于参数的梯度。

**提醒**：
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center> **图 3** : *线性->RELU->线性->SIGMOID* 的前向和反向传播<br>*紫色块表示前向传播，红色块表示反向传播。*</center></caption>

对于那些精通微积分的人（做这个作业不需要你是专家），可以使用微积分的链式法则导出二层网络中损失 \(\mathcal{L}\) 相对于 \(z^{[1]}\) 的导数，如下所示：

$$\frac{d \mathcal{L}(a^{[2]}, y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]}, y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

为了计算梯度 $dW^{[1]} = \frac{\partial \mathcal{L}}{\partial W^{[1]}}$，你使用上面的链式法则，并执行 $dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$。在反向传播期间，每一步你都会将当前梯度乘以对应层的梯度，以获取你想要的梯度。

同样，为了计算梯度 $db^{[1]} = \frac{\partial \mathcal{L}}{\partial b^{[1]}}$，你使用上面的链式法则，并执行 $db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$。

这就是为什么我们讨论**反向传播**的原因。

现在，类似于前向传播，您将在三个步骤中构建反向传播：
- 线性反向
- 线性 -> 激活反向，其中激活计算 ReLU 或 sigmoid 激活的导数
- [线性 -> RELU] \( \times \) (L-1) -> 线性 -> SIGMOID 反向（整个模型）

现在你将为整个网络实现反向传播函数。回想一下，当你实现了 `L_model_forward` 函数时，你在每次迭代时都存储了一个包含 (X,W,b, 和 z) 的缓存。在反向传播模块中，你将使用这些变量来计算梯度。因此，在 `L_model_backward` 函数中，你将从第 \(L\) 层开始，向后遍历所有隐藏层。在每一步中，你将使用第 \(l\) 层的缓存值通过第 \(l\) 层反向传播。下面的图 5 显示了反向传播的过程。

<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center> **图 5**：反向传播 </center></caption>

** 初始化反向传播**：
为了通过这个网络反向传播，我们知道输出是
$A^{[L]} = \sigma(Z^{[L]})$。因此，你的代码需要计算 $dAL= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$。
为此，使用这个公式（使用微积分导出，不需要深入了解）：

```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # # derivative of cost with respect to AL
```


然后，你可以使用这个激活后的梯度 $dAL$ 继续向后传播。如图 5 所示，你现在可以将 $dAL$ 输入到你实现的 LINEAR->SIGMOID 反向传播函数中（这将使用由 L_model_forward 函数存储的缓存值）。之后，你将需要使用一个 `for` 循环通过 LINEAR->RELU 反向传播函数遍历所有其他层。你应该将每个 $dA$, $dW$ 和 $db$ 存储在 grads 字典中。为此，使用以下公式：

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

例如，对于 $l=3$，这将在 `grads["dW3"]` 中存储 $dW^{[l]}$。

**练习**：为 *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* 模型实现反向传播。请您实现`layers.py`中的 ` L_model_backward` 函数。

In [ ]:
from layers import L_model_backward
from test_cases import L_model_backward_test_case, print_grads

AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

**预期得到的输出**:
    
```
dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]
```

# 5 - 损失函数
现在你将实现前向传播和反向传播。你需要计算损失函数，因为你想检查你的模型是否真的在学习。

使用以下公式计算交叉熵损失：

$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$

**练习**：请您实现`loss.py`中的 `cross_entropy_loss` 函数。


In [ ]:
from loss import cross_entropy_loss
from test_cases import cross_entropy_loss_test_case

Y, AL = cross_entropy_loss_test_case()

print("loss = " + str(cross_entropy_loss(AL, Y)))

**预期得到的输出**:
    
```
loss = 0.41493159961539694
```

# 6 - 初始化

你将编写两个辅助函数来初始化模型的参数。第一个函数将用于初始化两层模型的参数。第二个将把这个初始化过程推广到 \( L \) 层。

## 6.1 - 两层神经网络

**练习**：创建并初始化两层神经网络的参数。请您实现`initial.py`中的 `initialize_parameters` 函数。

**说明**：
- 模型的结构是：*线性 -> ReLU -> 线性 -> Sigmoid*。
- 对权重矩阵使用随机初始化。使用 `np.random.randn(shape)*0.01` 并给出正确的形状。
- 对偏置使用零初始化。使用 `np.zeros(shape)`。

In [ ]:
from initial import initialize_parameters

parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**预期得到的输出**:
    
```
W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[ 0.]
 [ 0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[ 0.]]
```

### 3.2 - L层神经网络

一个更深层次的L层神经网络的初始化更加复杂，因为有更多的权重矩阵和偏置向量。在完成 `initialize_parameters_deep` 时，你应该确保每一层之间的维度是匹配的。回想一下 $ n^{[l]} $ 是第 $ l $ 层中的单元数量。因此，例如如果我们输入 $ X $ 的大小是 $ (12288, 209) $（有 $ m=209 $ 个样本），那么：

|  |  **Shape of W** | **Shape of b** |  **Activation** | **Shape of Activation** |
|  :----: |  :---- |  :---- |  :---- |  :---- |
| **Layer 1** |  $(n^{[1]},12288)$ | $(n^{[1]},1)$  | $Z^{[1]} = W^{[1]}  X + b^{[1]}$ | $(n^{[1]},209)$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |
| **Layer 1** |  $A$ | $A$  | $A$ | $A$ |


Remember that when we compute $W X + b$ in python, it carries out broadcasting. For example, if: 

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Then $WX + b$ will be:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**练习**：为一个L层神经网络实现初始化。请您实现`initial.py`中的 `initialize_parameters_deep` 函数。

**指导**：
- 模型的结构是*[线性 -> ReLU] $ \times$ (L-1) -> 线性 -> Sigmoid*。也就是说，它有$L-1$层使用ReLU激活函数，然后是使用sigmoid激活函数的输出层。
- 对权重矩阵使用随机初始化。使用`np.random.randn(shape) * 0.01`。
- 对偏置使用零初始化。使用`np.zeros(shape)`。
- 我们将在变量`layer_dims`中存储$n^{[l]}$，即不同层中的单元数。例如，上周“平面数据分类模型”的`layer_dims`应该是[2,4,1]：有两个输入，一个具有4个隐藏单元的隐藏层，以及一个具有1个输出单元的输出层。这意味着`W1`的形状是(4,2)，`b1`是(4,1)，`W2`是(1,4)而`b2`是(1,1)。现在你将这个概念推广到$L$层！
- 这是$L=1$情况（一个层神经网络）的实现。它应该能激发你实现一般情况（L层神经网络）。

```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [ ]:
from initial import initialize_parameters_deep

parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**预期得到的输出**:
    
```
W1 = [[ 0.72642933 -0.27358579 -0.23620559 -0.47984616  0.38702206]
 [-1.0292794   0.78030354 -0.34042208  0.14267862 -0.11152182]
 [ 0.65387455 -0.92132293 -0.14418936 -0.17175433  0.50703711]
 [-0.49188633 -0.07711224 -0.39259022  0.01887856  0.26064289]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.55030959  0.57236185  0.45079536  0.25124717]
 [ 0.45042797 -0.34186393 -0.06144511 -0.46788472]
 [-0.13394404  0.26517773 -0.34583038 -0.19837676]]
b2 = [[0.]
 [0.]
 [0.]]
```

# 7 - 梯度下降

在这一部分，你将使用梯度下降更新模型的参数：

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

其中 $\alpha$ 是学习率。在计算出更新后的参数后，将它们存储在参数字典中。

**练习**：请您实现`optim.py`中的 `gradient_descent` 函数，使用梯度下降更新你的参数。

**指导**：
对每个 $W^{[l]}$ 和 $b^{[l]}$ 使用梯度下降进行更新，其中 $l = 1, 2, ..., L$。

In [ ]:
from optim import gradient_descent
from test_cases import gradient_descent_test_case

parameters, grads = update_parameters_test_case()
parameters = gradient_descent(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

**预期得到的输出**:
    
```
W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
```

# 8 - 总结

恭喜您成功实现了构建深度神经网络所需的所有函数！

我们知道这是一个漫长的任务，但未来的任务只会变得更好。下一个任务的难度会降低。

在下一个任务中，您将把所有这些函数结合起来构建两个模型：

- 一个两层神经网络
- 一个 L 层神经网络

请用Jupyter Notebook打来 `Application.ipynb`进入下一个lab，您将使用这些模型来对猫和非猫图像进行分类！